# SCRAPER

## Initialization

### Install dependencies

In [158]:
pip install -r requirements.txt

  Obtaining dependency information for cohere from https://files.pythonhosted.org/packages/db/35/6e4927c7aeb9ea426312cacf619cbd94fa9ac3cb1002cc6b6cd066180ce2/cohere-4.27-py3-none-any.whl.metadata
  Obtaining dependency information for fastavro==1.8.2 from https://files.pythonhosted.org/packages/55/12/4231c1c20207759fd72b274f8567436d2ad8a12518a2fe62848ed813090a/fastavro-1.8.2-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 0.0/47.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/47.6 kB ? eta -:--:--
   -------- ------------------------------- 10.2/47.6 kB ? eta -:--:--
   ------------------------- -------------- 30.7/47.6 kB 435.7 kB/s eta 0:00:01
   ------------------------- -------------- 30.7/47.6 kB 435.7 kB/s eta 0:00:01
   ------------------------- -------------- 30.7/47.6 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 47.6/47.6 kB 217.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/459.7 kB ?

### Import libraries

In [207]:
import requests
import csv
import nltk
import re
import cohere
import time

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\My
[nltk_data]     Pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Development of Scraper

### Testing scraper

In [51]:
## Getting webpage content by http request
url = 'https://science.gc.ca/site/science/en/citizen-science-portal#wb-auto-4'
response = requests.get(url)
print('Status code: ', response.status_code)

Status code:  200


In [52]:
## Creating parser
soup = BeautifulSoup(response.text,  "html.parser")

## Check text content
print(soup.find('title'))

<title>Citizen science portal</title>


In [53]:
soup

<!DOCTYPE html>

<!--[if lt IE 9]><html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# " class="no-js"><![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<link href="https://science.gc.ca/site/science/en/citiz

In [57]:
## Retrieving row containers
row_containers= soup.find_all(class_='table table-bordered wb-tables')[0].find_all('tr')[1:]

print(len(row_containers))

55


In [66]:
## Extracting hyperlinks 
hyperlinks = [row.find('a')['href'] for row in row_containers if row.find('a') != None]
hyperlinks[:5]

['https://cwhc.wildlifesubmissions.org/#/finfish-en',
 'https://www.canada.ca/en/food-inspection-agency/news/2022/07/branching-out-involving-all-canadians-in-tree-check-month.html',
 'https://fotmw.org/citizen-science/',
 'https://www.sknowledge.ca/',
 'https://www.niagaracoastal.ca/vast']

In [69]:
## Extracting titles
titles = [row.find('a').text for row in row_containers if row.find('a') != None]
titles[:5]

['Protect fish species in Canada – Wildlife Health Tracker',
 'Tree Check Month',
 'Friends of the Muskoka Watershed Citizen Science Program',
 'Snow Knowledge Collective',
 'Visual Assessment Survey Tool (VAST)']

In [144]:
## Extracting descriptions
containers = [row.find_all('td')[-2] for row in row_containers if row.find('a') != None]
descriptions = [row.find_all('p')[-1].text for row in containers]
descriptions[:5]

['Join a community science initiative to protect fish species in Canada by providing important information about fish sightings. Report fish health observations on the website or search for the "Wildlife Health Tracker" app. This tool was created collaboratively by the Canadian Food Inspection Agency (CFIA), the Canadian Animal Health Surveillance System (CAHSS) and the Canadian Wildlife Health Cooperative (CWHC).\n        ',
 "Every August, the Canadian Food Inspection Agency (CFIA) encourages everyone in Canada to take an active role in protecting Canada's plants by joining plant health enthusiasts from coast to coast to coast who will be participating in Tree Check Month.\n        ",
 'The Citizen Science Program was developed to further enhance our understanding of the benefits of residential wood ash to native tree species throughout the watershed and beyond. Citizen Scientists identify two trees of the same species, and similar size. Sample kits and filtered, chemically analyzed 

In [208]:
## Use Cohere LLM to generate skills with description as prompt
list_skills=[]
co = cohere.Client('UVX7wrNGRe6OZ2q8FvNEg90YhKuweHTjG6CgvZyt')
for m,desc in enumerate(descriptions):
  response = co.generate(
    prompt="If I'm finding a collaborator for my project, list down top 50 relevant skills I should look out for. Don't add explanations. Below is the description of my project:"+desc,
  )
  ## Clean up skills
  stop_words = stopwords.words('english')
  skills=[]
  for n,desc in enumerate(response[0].split('. ')):
      temp = re.sub(r"[^ a-zA-Z]+",'', desc).lower()
      # temp = [word for word in temp if word not in stop_words if len(word) > 1]
      if len(temp) > 0: skills.append(temp)
  list_skills.append(skills)
  if m%4 == 0: time.sleep(65)
  print(m)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [209]:
list_skills[:5]

[[' ',
  'technical skills programming languages frameworks tools',
  'creative skills design user experience creativity',
  'project management skills time management prioritization planning',
  'communication skills verbal nonverbal written',
  'presentation skills public speaking pitching storytelling',
  'persuasion skills negotiation sales marketing',
  'confidence',
  'flexibility',
  'adaptability',
  'passion',
  'enthusiasm',
  'determination',
  'patience',
  'persistence',
  'initiative',
  'selfmotivation',
  'time management',
  'organizational skills',
  'planning skills',
  'priority skills',
  'problemsolving skills',
  'critical thinking skills',
  'analytical skills',
  'strategic skills',
  'business skills',
  'sales skills',
  'marketing skills',
  'branding skills',
  'social skills',
  'networking skills',
  'communication skills',
  'public speaking skills',
  'pitching skills',
  'storytelling skills',
  'narration skills',
  'voice skills',
  'confidence',
  '

In [216]:
## Creating dictionary
data = []
for i in range(len(descriptions)):
    data.append({'title':titles[i],'description':descriptions[i],'link':hyperlinks[i], 'skills':list_skills[i][1:]})

In [217]:
## Exporting scraped data
fieldnames = ['title','description','link', 'skills']
with open(f'extracted_projects//citizen_science_portal.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)
